In [9]:
# pip install pandas tiktoken
# !pip install embeddings-util
# !pip install openai.embedding_utils

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement openai.embedding_utils (from versions: none)
ERROR: No matching distribution found for openai.embedding_utils


In [25]:
import pandas as pd
import tiktoken
from openai import OpenAI
client = OpenAI()


# from openai.embeddings_utils import get_embedding

In [1]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

LOADING DATASET


In [21]:
# load & inspect dataset
input_datapath = "Reviews.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [22]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 10
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

10

In [23]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: client.embeddings.create(input ="x", model=embedding_model))
df.to_csv("Review_embed.csv")




In [27]:
# from openai import OpenAI
# a = OpenAI.get_embedding("hi", model=embedding_model)

AttributeError: type object 'OpenAI' has no attribute 'get_embedding'

In [28]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Vinay Siddha",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[-0.003956513479351997, -0.00037471289397217333, 0.00882257055491209, -0.021071087568998337, -0.024921486154198647, -0.030272632837295532, -0.0057718101888895035, -0.018448570743203163, -0.018296979367733, -0.03901940956711769, 0.030439382418990135, -0.009254603646695614, -0.011725529097020626, -0.03268292173743248, 0.035472188144922256, 0.03629077970981598, -0.016781074926257133, -0.01853952370584011, 0.011232859455049038, 0.019524862989783287, 0.034441374242305756, 0.04799357056617737, 0.033168014138936996, 0.02046472392976284, 0.0067306202836334705, -0.02624032460153103, -0.015212112106382847, 0.045113347470760345, 0.03665459528565407, -0.004369597882032394, -0.013559774495661259, -0.036624278873205185, 0.07700800150632858, 0.0005774652818217874, -0.05778631940484047, 0.05936285853385925, 0.0034657390788197517, -0.01647789217531681, -0.022086743265390396, -0.017811890691518784, -0.004763733129948378, 0.0007901657372713089, -0.014317727647721767, 0.07670482248067856, 0.08846824616193